In [16]:
import logging
from typing import Dict, List
import requests
import pandas as pd
from time import sleep

from configparser import ConfigParser

pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
#pd.set_option('display.max_rows', None)  # Mostrar todas las filas

In [17]:
config = ConfigParser()
config.read("C:/envs/riot-env/riot-games-api/config/config.ini")

params: Dict[str, str] = {
    "base_url_region": config["api_riot"]["base_url_region"],
    "base_url_server": config["api_riot"]["base_url_server"],
    "API_KEY": config["api_riot"]["API_KEY"],
#---Información del jugador---
    "player_name": config["player_info"]["player_name"],
    "player_id": config["player_info"]["player_id"],
    "player_puuid": config["player_info"]["player_puuid"],
    "region": config["player_info"]["region"],
    "servidor": config["player_info"]["servidor"],
#---Endpoints---
    "MATCH-V5": config["endpoint_url"]["MATCH-V5"],
    "MATCHES-V5": config["endpoint_url"]["MATCHES-V5"]
}

""" logging.basicConfig(filename='/home/leo/env/riot-env/riot-games-api/app.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
 """
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [18]:
#startTime=1704895200&endTime=1714572000&queue=420&type=ranked&start=0&count=100
def get_matches_history(start_time: int, end_time: int, queue: int, match_type: str, start: int, batch_size: int) -> List[str]:

    match_ids: List[str] = []

    logging.info('Obteniendo historial de partidas...')

    try:
        while True:
            r_url_matches = f"{params['base_url_region']}/{params['MATCH-V5']}/{params['player_puuid']}/ids?start_time={start_time}&end_time={end_time}&queue={queue}&type={match_type}&start={start}&count={batch_size}&api_key={params['API_KEY']}"

            r = requests.get(r_url_matches)

            if r.status_code == 200:
                batch_match_ids = r.json()

                if not batch_match_ids or len(batch_match_ids) < batch_size:
                    logging.info('No se encontraron más partidas.')
                    break

                match_ids += batch_match_ids
                start += batch_size
            else:
                logging.error(f"Error: {r.status_code}, {r.content}")
                break

        logging.info('Se obtuvo exitosamente el historial de partidas.')

    except requests.exceptions.RequestException as e:
        logging.error(f"Error al hacer la solicitud: {e}")
    except Exception as e:
        logging.error(f"Error inesperado: {e}")

    return match_ids

In [21]:
def obtener_informacion_partidas(player_matches_ids: List[str]) -> List[Dict]:

    match_details_list: List[Dict] = []

    logging.info('Extrayendo estadísticas...')

    for match_id in player_matches_ids:
        try:
            r_url_matches_info = f"{params['base_url_region']}/{params['MATCHES-V5']}/{match_id}?api_key={params['API_KEY']}"

            r = requests.get(r_url_matches_info)

            if r.status_code == 200:
                player_matches_info = r.json()

                # Encuentra el índice del jugador en la partida
                part_index = player_matches_info['metadata']['participants'].index(params['player_puuid'])

                # Filtra las estadísticas relevantes del jugador en la partida
                filtred_stats = {
                    
                    'matchId': player_matches_info['metadata']['matchId'],
                    'gameId': player_matches_info['info']['gameId'],
                    'gameCreation': player_matches_info['info']['gameCreation'],
                    'gameStartTimestamp': player_matches_info['info']['gameStartTimestamp'],
                    'gameEndTimestamp': player_matches_info['info']['gameEndTimestamp'],
                    'gameDuration': player_matches_info['info']['gameDuration'],
                    'gameEndedInEarlySurrender': player_matches_info['info']['participants'][part_index]['gameEndedInEarlySurrender'],
                    'win': player_matches_info['info']['participants'][part_index]['win'],

                    'individualPosition': player_matches_info['info']['participants'][part_index]['individualPosition'],
                    'lane': player_matches_info['info']['participants'][part_index]['lane'],
                    'championId': player_matches_info['info']['participants'][part_index]['championId'],
                    'championName': player_matches_info['info']['participants'][part_index]['championName'],
                    'champLevel': player_matches_info['info']['participants'][part_index]['champLevel'],

                    'kills': player_matches_info['info']['participants'][part_index]['kills'],
                    'deaths': player_matches_info['info']['participants'][part_index]['deaths'],
                    'assists': player_matches_info['info']['participants'][part_index]['assists'],
                    'kda': player_matches_info['info']['participants'][part_index]['challenges']['kda'],
                    'killParticipation': player_matches_info['info']['participants'][part_index]['challenges'].get('killParticipation', None),
                    'doubleKills': player_matches_info['info']['participants'][part_index]['doubleKills'],
                    'tripleKills': player_matches_info['info']['participants'][part_index]['tripleKills'],
                    'quadraKills': player_matches_info['info']['participants'][part_index]['quadraKills'],
                    'pentaKills': player_matches_info['info']['participants'][part_index]['pentaKills'],

                    'abilityUses': player_matches_info['info']['participants'][part_index]['challenges']['abilityUses'],
                    'goldEarned': player_matches_info['info']['participants'][part_index]['goldEarned'],
                    'goldSpent': player_matches_info['info']['participants'][part_index]['goldSpent'],
                    'goldPerMinute': player_matches_info['info']['participants'][part_index]['challenges']['goldPerMinute'],

                    'totalMinionsKilled': player_matches_info['info']['participants'][part_index]['totalMinionsKilled'],
                }

                sleep(1)
                match_details_list.append(filtred_stats)
            else:
                logging.error(f"URL MATCHES INFO ERROR: {r.status_code}, {r.content}")

        except Exception as e:
            logging.error(f"Error al procesar la partida {match_id}: {e}")

    return match_details_list

In [22]:
get_matches_history(1704895200, 1714572000, 420, "ranked", 0, 100)
id_matches = get_matches_history(1704895200, 1714572000, 420, "ranked", 0, 100)
match_details = obtener_informacion_partidas(id_matches)

2024-05-07 09:45:55,570 - INFO - Obteniendo historial de partidas...
2024-05-07 09:46:03,898 - INFO - No se encontraron más partidas.
2024-05-07 09:46:03,899 - INFO - Se obtuvo exitosamente el historial de partidas.
2024-05-07 09:46:03,901 - INFO - Obteniendo historial de partidas...
2024-05-07 09:46:11,930 - INFO - No se encontraron más partidas.
2024-05-07 09:46:11,931 - INFO - Se obtuvo exitosamente el historial de partidas.
2024-05-07 09:46:11,933 - INFO - Extrayendo estadísticas...


In [23]:
df = pd.DataFrame(match_details)
df

,matchId,gameId,gameCreation,gameStartTimestamp,gameEndTimestamp,gameDuration,gameEndedInEarlySurrender,win,individualPosition,lane,championId,championName,champLevel,kills,deaths,assists,kda,killParticipation,doubleKills,tripleKills,quadraKills,pentaKills,abilityUses,goldEarned,goldSpent,goldPerMinute,neutralMinionsKilled,totalAllyJungleMinionsKilled,totalEnemyJungleMinionsKilled,totalMinionsKilled
0,LA2_1409189580,1409189580,1715082720412,1715082800753,1715084741496,1940,False,True,BOTTOM,BOTTOM,67,Vayne,17,11,10,11,2.200000,0.536585,2,0,0,0,198,15689,23453,485.015472,4,4,0,204
1,LA2_1409186321,1409186321,1715077867839,1715077950228,1715079646315,1696,False,True,UTILITY,BOTTOM,16,Soraka,13,1,6,28,4.833333,0.805556,0,0,0,0,201,9397,8275,332.399907,0,0,0,23
2,LA2_1409183989,1409183989,1715072736782,1715072774016,1715074264872,1490,False,True,UTILITY,BOTTOM,350,Yuumi,14,2,3,25,9.000000,0.675000,0,0,0,0,301,8623,6850,347.038371,0,0,0,11
3,LA2_1409179848,1409179848,1715068551680,1715068650215,1715070545889,1895,False,True,UTILITY,JUNGLE,40,Janna,14,1,4,25,6.500000,0.896552,0,0,0,0,249,9888,8575,312.956428,0,0,0,19
4,LA2_1409178659,1409178659,1715063556319,1715063633184,1715065114931,1481,False,False,TOP,TOP,84,Akali,14,8,8,7,1.875000,0.625000,0,0,0,0,227,10968,10700,444.087651,0,0,0,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,LA2_1353285330,1353285330,1699010719584,1699010818982,1699012611714,1792,False,False,UTILITY,BOTTOM,61,Orianna,12,2,6,7,1.500000,0.375000,0,0,0,0,339,7654,7135,256.171999,0,0,0,33
696,LA2_1353281714,1353281714,1699003710622,1699003756287,1699005322808,1566,False,True,MIDDLE,MIDDLE,246,Qiyana,14,5,4,12,4.250000,0.548387,1,0,0,0,282,11074,10900,424.122029,10,0,7,156
697,LA2_1353283266,1353283266,1699003019057,1699003076207,1699003180009,103,True,True,UTILITY,NONE,40,Janna,1,0,0,0,0.000000,NaN,0,0,0,0,2,500,500,288.453988,0,0,0,0
698,LA2_1353281316,1353281316,1698999016587,1698999043886,1699000841246,1797,False,False,MIDDLE,MIDDLE,84,Akali,15,8,8,10,2.250000,0.642857,0,0,0,0,276,11766,11700,392.752404,0,0,0,165
